CA02: This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

IMPORTANT NOTE:

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.

In [1]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...
from sklearn.naive_bayes import GaussianNB ## This module facilitates the training of a Gaussian Naive Bayes model using features (x) and target (y), and it includes a prediction function.
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics # Importing the metrics module from scikit-learn for evaluating and measuring model performance.

import pandas as pd

In [2]:
def make_Dictionary(root_dir): # Create a function that accepts a directory path as an argument

  all_words = []                                                        # 1. Create an empty list to store all the words extracted from our email inputs.
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]    # 2. Retrieve each file name in the specified folder (e.g., train- and test-email folders) using the os module for file and directory operations.

  for mail in emails:               # 1. Use a for loop to iterate over each email file, equivalent to saying "for each mail in our emails."
    with open(mail) as m:           # 2. The "open()" function opens a file, and "as m" renames it as 'm'. The "open()" function takes three arguments: open(filename, mode, encoding).
      for line in m:                # 3. Use a for loop to iterate over each line in opened email (iterate over each line, one email at a time).
        words = line.split()        # 4. Divide each line into individual words and store them in the variable "words."
        all_words += words          # 5. Gathers all the recorded "words" and compiles them into a list named "all_words."

  dictionary = Counter(all_words)     # 1. Utilizing the Counter() function from the collections module, the "all_words" list is converted into a dictionary. Specifically, each distinct word becomes a key, and its occurrences are recorded as the corresponding value in the key-value pair
  list_to_remove = list(dictionary)   # 2. Change the dictionary into a list to facilitate iteration, enabling the elimination of undesirable characters and words


  for item in list_to_remove:     # 1. Utilize a for loop to traverse through each element in the recently created list.
    if item.isalpha() == False:   # 2. The ".isalpha()" method verifies whether the item comprises exclusively alphabetical letters. It yields True for alphabetic items and False otherwise.
      del dictionary[item]        # 3. As we're inspecting for non-alphabetical items (as determined in the line above), deletion of the item occurs only when it includes a non-alphabetical character.
    elif len(item) == 1:          # 4. When the item comprises only alphabetical characters, we check if it is a single character
      del dictionary[item]        # 5. If it happens to be a single character, we proceed to remove it.


  dictionary = dictionary.most_common(3000)   # 1. The ".most_common()" function, part of the collections library, retains solely the top 3,000 (as specified) most frequently occurring words in the dictionary based on their respective frequencies.
  return dictionary                           # 2. Provide the dictionary as the output



In [3]:
def extract_features(mail_dir):       # Create a function that takes a directory path as its input.

  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]    # 1. Utilizing "os.path.join()", this function intelligently concatenates multiple paths using "/" as a separator. For more information, refer to the documentation: https://www.geeksforgeeks.org/python-os-path-join-method/
  features_matrix = np.zeros((len(files),3000))                       # 2. In this context, "features" exclusively denotes Independent Variables (IVs) and excludes the target (Dependent Variable, DV). Utilizing the "np.zeros" function, an array is created with a 2D shape determined by the length of files and 3,000, filled with zeros. More information can be found in the documentation: https://www.geeksforgeeks.org/numpy-zeros-python/
  train_labels = np.zeros(len(files))                                 # 3. Using a single argument for "np.zeros" results in a 1D array filled with zeros, assigning train_labels as such. In this setup, 0 signifies NOT SPAM, while 1 denotes SPAM.
  count = 1;                                                          # 4. Set up a counter variable named "count" with an initial value.
  docID = 0;

  for fil in files:                                                     # 1. Use a for loop with the variable "fil" to iterate through each intelligently joined file in the "files" collection.
    with open(fil) as fi:                                               # 2. The "open()" function is used to open a file, and it is given the alias 'fi'. The "open()" function takes three arguments: filename, mode, an
      for i, line in enumerate(fi):                                     # 3. Go through each line in the file, keeping track of the line number (index i) and the actual content of the line (line).
        if i ==2:                                                       # 4. Check if the current line
          words = line.split()                                          # 5. Split the line into individual words, employing whitespace as the separator.
          for word in words:                                            # 6. Iterate over each word in the list of words
            wordID = 0                                                  # 7. Create a variable to store the ID of the current word in the dictionary
            for i, d in enumerate(dictionary):                          # 8. Loop through the dictionary using enumerate to access the index (i) and the word-frequency pair (d)
              if d[0] == word:                                          # 9. Verify if the word in the dictionary matches the current word
                wordID = i                                              # 10. If there's a match, set wordID to the index in the dictionary
                features_matrix[docID,wordID] = words.count(word)       # 11. Update the feature matrix with the count of the current word in the current document

      train_labels[docID] = 0;                            # Initialize the label for the current document as 0
      filepathTokens = fil.split('\\')                     # Split the file path into tokens using the '/' separator
      lastToken = filepathTokens[len(filepathTokens)-1]   # Get the last token from the file path
      if lastToken.startswith("spmsg"):                   # Check if the last token starts with "spmsg"
        train_labels[docID] = 1;                          # If it does, set the label for the current document to 1
        count = count + 1                                 # Increment the count
      docID = docID + 1                                   # Increment the document ID for the next iteration
  return features_matrix, train_labels                    # Return the resulting feature matrix and labels

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
TRAIN_DIR = r"/content/drive/MyDrive/Data/train-mails"
TEST_DIR = r"/content/drive/MyDrive/Data/test-mails"

In [6]:
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from files.")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)


reading and processing emails from files.


In [7]:
# In this section enter your code to TRAIN the model using Naive Bayes algorithm, then PREDICT and then evaluate PERFORMANCE (Accuracy)
# Your code below ...
# Create an instance of a Gaussian Naive Bayes classifier and assign it to nb_classifier
# instantiate a Gaussian NB classifier = nb_classifier

    # 8 SPAM emails are incorrectly labeled as NOT SPAM whereas 1 NOT SPAM is incorrectly labeled as SPAM
    # Display the confusion matrix in table format



# Creating and training a Gaussian Naive Bayes model
email_spam_model = GaussianNB()
train = email_spam_model.fit ( features_matrix, labels)
print("training Model Gaussian Naibe Bayes Algorithm...")
print("Trainin completed")


# Making predictions on the test set
test = email_spam_model.predict(test_features_matrix)
print("testing trained model to predict test data labes... ")
print("testing completed")


# Calculating and displaying accuracy
accuracy = accuracy_score ( test_labels, test)
print("Accuracy of the model is: ", accuracy)


# Your output should look like below if your code is right

training Model Gaussian Naibe Bayes Algorithm...
Trainin completed
testing trained model to predict test data labes... 
testing completed
Accuracy of the model is:  1.0


======================= END OF PROGRAM =========================